## Main code

In [1]:
#%reset

In [2]:
import os, sys, glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np

import mne

import matplotlib
import matplotlib.pyplot as plt
import time

from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

from tools.train import eval_model_cl
from tools.data import unfold_raw, rm_repeated_annotations


cpu


## Read Data

In [3]:
subj_ids = [1] #list(range(1, 20))
fs = 128
window_size = int(5 * fs)
stride_size_train, stride_size_val, stride_size_test = int(2.5 * fs), int(5 * fs), int(5 * fs)
n_channs = 129 # 128 for eeg, 1 for env
batch_size = int(32)
print('-------------------------------------')
print(f'window_size: {window_size}  stride_size_test: {stride_size_test}')

dataset_name = ['rochester_data', 'natural_speech']
outputs_path = f'../outputs/'
data_path = os.path.join(outputs_path, dataset_name[0], dataset_name[1])
after_ica_path = os.path.join(data_path, 'after_ica_raw')
print(f'data_path: {data_path}')

-------------------------------------
window_size: 640  stride_size_test: 640
data_path: ../outputs/rochester_data/natural_speech


In [4]:
def load_data(subject_ids, subjects_dir):

    raws_train_windowed, raws_val_windowed, raws_test_windowed = [], [], []

    for subj_id in subj_ids:
        

        # load subject raw MNE object
        raw = mne.io.read_raw(os.path.join(after_ica_path, f'subj_{subj_id}_after_ica_raw.fif'), preload=True)
        # drop M1 and M2 channels
        raw.drop_channels(['M1', 'M2'])
        assert raw.info['nchan'] == n_channs

        raw = rm_repeated_annotations(raw)
        annots = raw.annotations.copy()
        raw_split = [raw.copy().crop(t1, t2) for t1, t2 in zip(annots.onset[:-1]+annots.duration[:-1], annots.onset[1:])]

        # Pick the split with the longest duration for validation, supposedly less noisy
        ix_val = np.argmax([i.get_data().shape[1] for i in raw_split])
        raw_val = [raw_split.pop(ix_val)] # create a list to make it iterable. later may be used for multiple splits

        # Pick the next split with the longest duration for testing, supposedly less noisy
        ix_test = np.argmax([i.get_data().shape[1] for i in raw_split])
        raw_test = [raw_split.pop(ix_test)]
        
        # creat list of unfolded tensor raw objects
        fs = raw.info['sfreq']
        raws_train_windowed.extend([unfold_raw(i, window_size=window_size, stride=stride_size_train) for i in raw_split if i.get_data().shape[1] > window_size])
        raws_val_windowed.extend([unfold_raw(i, window_size=window_size, stride=stride_size_val) for i in raw_val if i.get_data().shape[1] > window_size])
        raws_test_windowed.extend([unfold_raw(i, window_size=window_size, stride=stride_size_test) for i in raw_test if i.get_data().shape[1] > window_size])
        print("-------------------------------------")
        print('N train: %d  N val: %d  N test: %d' % (len(raws_train_windowed), len(raws_val_windowed), len(raws_test_windowed)))

    # concatenate all in second dimension
    sigs_train = torch.cat(raws_train_windowed, dim=1).permute(1, 0, 2, 3)
    sigs_val = torch.cat(raws_val_windowed, dim=1).permute(1, 0, 2, 3)
    sigs_test = torch.cat(raws_test_windowed, dim=1).permute(1, 0, 2, 3)
    print(f"Shape Trian: {sigs_train.shape}  Shape Val: {sigs_val.shape}  Shape Test: {sigs_test.shape}")

    eegs_train = sigs_train[:, :, :-1, :]
    eegs_val = sigs_val[:, :, :-1, :]
    eegs_test = sigs_test[:, :, :-1, :]
    print("-------------------------------------")
    print(f"Shape EEG Train: {eegs_train.shape}  Val: {eegs_val.shape}  Test: {eegs_test.shape}")

    # To avoid information leakage, we estimate the mean and std from the training set only.
    mean_eeg_train =  eegs_train.mean()
    std_eeg_train = eegs_train.std()
    print(f"Mean: {mean_eeg_train}  Std: {std_eeg_train}")

    envs_train = sigs_train[:, :, [-1], :]
    envs_val = sigs_val[:, :, [-1], :]
    envs_test = sigs_test[:, :, [-1], :]
    print(f"Shape Env Train: {envs_train.shape}  Val: {envs_val.shape}  Test: {envs_test.shape}")

    # Estimate mean and std of the Envelope data set
    mean_env_train =  envs_train.mean()
    std_env_train = envs_train.std()
    print(f"Mean Env: {mean_env_train}  Std Env: {std_env_train}")

    # Normalize the data
    eegs_train = (eegs_train - mean_eeg_train) / std_eeg_train
    eegs_val = (eegs_val - mean_eeg_train) / std_eeg_train
    eegs_test = (eegs_test - mean_eeg_train) / std_eeg_train

    envs_train = (envs_train - mean_env_train) / std_env_train
    envs_val = (envs_val - mean_env_train) / std_env_train
    envs_test = (envs_test - mean_env_train) / std_env_train

    return {'eegs_train': eegs_train, 'eegs_val': eegs_val, 'eegs_test': eegs_test, 
            'envs_train': envs_train, 'envs_val': envs_val, 'envs_test': envs_test}

X = load_data(subj_ids, after_ica_path)

Opening raw data file ../outputs/rochester_data/natural_speech/after_ica_raw/subj_1_after_ica_raw.fif...
    Range : 0 ... 464571 =      0.000 ...  3629.461 secs
Ready.
Reading 0 ... 464571  =      0.000 ...  3629.461 secs...
Initial num of annots: 48  Num of removed annots: 19  Num of retained annots:  29
-------------------------------------
N train: 26  N val: 1  N test: 1
Shape Trian: torch.Size([1204, 1, 129, 640])  Shape Val: torch.Size([36, 1, 129, 640])  Shape Test: torch.Size([36, 1, 129, 640])
-------------------------------------
Shape EEG Train: torch.Size([1204, 1, 128, 640])  Val: torch.Size([36, 1, 128, 640])  Test: torch.Size([36, 1, 128, 640])
Mean: 9.050915399100301e-12  Std: 5.065590812591836e-06
Shape Env Train: torch.Size([1204, 1, 1, 640])  Val: torch.Size([36, 1, 1, 640])  Test: torch.Size([36, 1, 1, 640])
Mean Env: 2.364877700805664  Std Env: 2.5979256629943848


### Pytorch dataloader

In [5]:
class MyDataset(Dataset):
    def __init__(self, eeg, env):
        self.eeg = eeg
        self.env = env
    
    def __getitem__(self, index):
        return self.eeg[index], self.env[index]
    
    def __len__(self):
        return len(self.eeg)
    


In [6]:
dataset_train = MyDataset(X['eegs_train'], X['envs_train'])
dl_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
dl_val = DataLoader(MyDataset(X["eegs_val"], X["envs_val"]), batch_size=batch_size, shuffle=True, drop_last=True)

## Model

In [7]:
import torch
from models.eeg_encoder import EEGEncoder
from models.envelope_encoder import EnvelopeEncoder
from models.contrastive_eeg_speech import CLEE
eeg_encoder = EEGEncoder()
env_encoder = EnvelopeEncoder()
model = CLEE(eeg_encoder, env_encoder)

# Test the model, add no grad
with torch.no_grad():
    print(eeg_encoder(X['eegs_train'][:32, :, :, :]).shape)

summary(eeg_encoder, (1, 128, 640))
summary(env_encoder, (1, 1, 640))
summary(model, [(1, 128, 640), (1, 1, 640)])

torch.Size([32, 320])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 128, 640]             520
       BatchNorm2d-2          [-1, 8, 128, 640]              16
            Conv2d-3           [-1, 16, 1, 640]           2,048
       BatchNorm2d-4           [-1, 16, 1, 640]              32
               ELU-5           [-1, 16, 1, 640]               0
         AvgPool2d-6           [-1, 16, 1, 160]               0
           Dropout-7           [-1, 16, 1, 160]               0
            Conv2d-8           [-1, 16, 1, 160]             256
            Conv2d-9           [-1, 16, 1, 160]             256
      BatchNorm2d-10           [-1, 16, 1, 160]              32
              ELU-11           [-1, 16, 1, 160]               0
        AvgPool2d-12            [-1, 16, 1, 20]               0
          Dropout-13            [-1, 16, 1, 20]               0
          Flatten

/Users/keyvan.mahjoory/opt/anaconda3/envs/mne/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525699189/work/aten/src/ATen/native/Convolution.cpp:896.)
  return F.conv2d(input, weight, bias, self.stride,


In [8]:
models_dict = {'SIMPLE': model}
lossi = []
udri = [] # update / data ratio 
ud = []

lr = 0.001

for name, model in models_dict.items():

    # Reset for the new model in the loop
    print(f"+--------------New model: {name}----------------------+")
    writer = SummaryWriter(log_dir=f"runs/{name}_{time.strftime('%Y%m%d_%H%M%S')}")
    model.to(device)
    optimizer = optim.NAdam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.7)
    cnt = 0
    loss_batches = []


    for epoch in range(1, 50):

        print(f"====== Epoch: {epoch}")

        model.train()
        for ix_batch, (Xb_eeg, Xb_env) in enumerate(dl_train):

            # send to device
            Xb_eeg = Xb_eeg.to(device)
            Xb_env = Xb_env.to(device)

            # Zero out gradients
            optimizer.zero_grad()

            # forward pass
            eeg_features, env_features, logit_scale = model(Xb_eeg, Xb_env) 


            # normalize features
            eeg_features_n = eeg_features / eeg_features.norm(dim=1, keepdim=True)
            env_features_n = env_features / env_features.norm(dim=1, keepdim=True)

            # logits
            logits_per_eeg = logit_scale * eeg_features_n @ env_features_n.t()
            logits_per_env = logits_per_eeg.t()

            #loss function
            labels = torch.arange(batch_size).to(device)
            loss_eeg = F.cross_entropy(logits_per_eeg, labels)
            loss_env = F.cross_entropy(logits_per_env, labels)
            loss   = (loss_eeg + loss_env)/2

            # backward pass
            loss.backward()
            optimizer.step()

            loss_batches.append(loss.item())
            cnt += 1

            with torch.no_grad():
                #ud = {f"p{ix}":(lr*p.grad.std() / p.data.std()).log10().item() for ix, p in enumerate(model.parameters()) if p.ndim==4 }
                #writer.add_scalars('UpdateOData/ud', ud, cnt)
                writer.add_scalar('Loss/train_batch', loss.item(), cnt)

            # normalize weights
            with torch.no_grad():
                model.eeg_encoder.normalize_weights()
            
            #break   

        loss_epoch = loss_batches[-(ix_batch + 1):]  # mean loss across batches
        loss_epoch = sum(loss_epoch) / len(loss_epoch)
        writer.add_scalar('Loss/train_epoch', loss_epoch, epoch)
        #for pname, p in model.named_parameters():
        #writer.add_histogram(f'Params/{pname}', p, epoch)
        #writer.add_histogram(f'Grads/{pname}', p.grad, epoch)

        loss_val, *_ = eval_model_cl(dl_val, model, device=device)
        writer.add_scalar('Loss/val_epoch', loss_val, epoch)

        

        model.train()

        # Update learning rate based on epoch
        scheduler.step()
            
    #break   


+--------------New model: SIMPLE----------------------+
====== Epoch: 1
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
====> Validation loss: 3.3886,  X1 loss: 3.3917   X2 loss: 3.3854
====== Epoch: 2
_____
_____
_____
_____
_____
_____
_____
_____
_____
_____
